In [ ]:
import duckdb
import xml.etree.ElementTree as ET
# Import classifier
from ipynb.fs.full.group_classifier import predict_batch
print('hi')

In [ ]:
# Connect to sql database
con = duckdb.connect(database='../data/database/german-parliament', read_only=False)

In [ ]:
def preprocess_speech(speech:tuple) -> tuple[int, list[str]]:
    # We only need the speech it to later identify the origin speech
    speech_id = test_speech[0]
    text = test_speech[5]
    # Parse xml from string will throw ParseError if not parseable
    root = ET.fromstring(text)
    # We only need paragraphs -> this filters unecessary information, such as comments or applause
    paragraphs = root.findall('p')
    paragraphs_text = {"paragraphs":[]}
    # Get text of paragraphs
    for p in paragraphs:
        # Use this to filter out speaker information
        if p.attrib.get("klasse") == "redner":
            continue
        else:
            paragraphs_text["paragraphs"].append(p.text.strip()) #get text and remove potential irrelevant whitespaces

    return (speech_id, paragraphs_text)
    

In [ ]:
test_speech = con.sql("select * from speech order by random() limit 1 ").fetchone()

In [ ]:
speech_id, paragraphs = preprocess_speech(test_speech)

print(speech_id)
print(paragraphs)

In [ ]:
print(ET.tostring(root.find('name'), encoding='unicode'))